In [1]:
import tdmclient.notebook
await tdmclient.notebook.start()

In [2]:
!python -m tdmclient list

id:         aa24ac7a-325b-4e3c-b43c-6e53947c2376
group id:   326a3569-fb02-4274-b780-bff2e9fe874f
product id: 8 (Thymio II)
name:       Thymio 308
status:     3 (busy)
cap:        7
firmware:   14



In [3]:
from tdmclient import ClientAsync, aw
import time
import math
import numpy as np

In [5]:
@tdmclient.notebook.sync_var 
def ground_sensor():
    global prox_ground_reflected
    return prox_ground_reflected

@tdmclient.notebook.sync_var 
def horiz_sensor():
    global prox_horizontal
    return prox_horizontal

@tdmclient.notebook.sync_var 
def print_sensor_values(sensor_func, print_duration=3, delta_time=0.5):
    """
    While the end time has not been reached, print the sensor values every delta_time seconds
    """
    
    t_end = time.time()+print_duration

    while time.time() < t_end:
        time.sleep(delta_time)
        print(sensor_func())

In [6]:
print_sensor_values(horiz_sensor)

[2530, 4717, 4378, 3313, 0, 0, 0]
[2535, 4714, 4378, 3314, 0, 0, 0]
[2529, 4714, 4375, 3312, 0, 0, 0]
[2525, 4715, 4375, 3308, 0, 0, 0]
[2528, 4714, 4380, 3309, 0, 0, 0]
[2526, 4714, 4377, 3309, 0, 0, 0]


In [ ]:
%%run_python

# Constants at the start of the program
state = 1                     # state of the navigation: "global"=1 or "local"=0
obst = [0, 0, 0, 0, 0]               # measurements of the front proximity sensors, not taking into account the back sensors
motor_speed = [0, 0]                 # motor speed of the robot [left, right]
sensor_scale = 200   # scale of the proximity sensors
obstThrL = 10        # low obstacle threshold to switch state local->global (equivalent to 2000)
obstThrH = 20        # high obstacle threshold to switch state global->local (equivalent to 4000)
obstSpeedGain = 5    # /100 (actual gain: 5/100=0.05)
initial_speed = 50  # initial speed of the robot
is_goal_reached = False   # flag to check if the goal is reached
max_speed = 500           # maximum speed of the robot for safety

w_l = [20,  10, -10, -10, -20] # weights for the left motor
w_r = [-20, -10, -10,  10,  20] # weights for the right motor


@onevent
def prox():
    global prox_horizontal, state, obst, w_l, w_r, motor_left_target, motor_right_target, sensor_scale, obstThrL, obstThrH, initial_speed, max_speed

    # Get the measurements from the proximity sensors
    obst = [prox_horizontal[0] // sensor_scale, prox_horizontal[1] // sensor_scale, prox_horizontal[2] // sensor_scale, prox_horizontal[3] // sensor_scale, prox_horizontal[4] // sensor_scale]

    # check if the state should be switched
    if state == 0:
        if obst[0] < obstThrL and obst[1] < obstThrL and obst[2] < obstThrL and obst[3] < obstThrL and obst[4] < obstThrL:
            state = 1
            print("Switching to global navigation: No detected obstacle anymore..")
    elif state == 1:
        if obst[0] > obstThrH or obst[1] > obstThrH or obst[2] > obstThrH or obst[3] > obstThrH or obst[4] > obstThrH:
            state = 0
            print("Switching to local navigation: Obstacle detected..")

    # Navigation
    if state == 1:
        motor_left_target = initial_speed
        motor_right_target = initial_speed

    elif state == 0:
        y = [0, 0]
        for i in range(len(obst)):
            y[0] = y[0] + w_l[i] * obst[i]
            y[1] = y[1] + w_r[i] * obst[i]
        print("Left motor speed: ", y[0])
        print("Right motor speed: ", y[1])
        motor_left_target = y[0]
        motor_right_target = y[1]

In [15]:
motor_left_target = 0
motor_right_target = 0

In [12]:
await tdmclient.notebook.stop()